In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import openpyxl
import datetime
import re
import string
import pytz
import requests

from date_extractor import extract_dates


# Process missing data extracted from SAA database

In [706]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')

athletes = pd.read_csv("wpdatatable_2.csv")

In [707]:
athletes

,wdt_ID,newcolumn,newcolumn1,newcolumn2,newcolumn3,team,result,windms,competition,yeardob,infoifany
0,1,2010-12-18,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.59,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia..."
1,2,2010-11-21,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.67,-0.1,International - Asian Games,1987,Open Heats 3 Guangdong Olympic Stadium
2,3,2010-12-18,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.69,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia..."
3,4,2010-11-21,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.69,1.2,International - Asian Games,1987,Open Heats 1 Guangdong Olympic Stadium
4,5,2010-11-22,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.73,0.7,International - Asian Games,1987,Open Semis 3 Guangdong Olympic Stadium
...,...,...,...,...,...,...,...,...,...,...,...
144117,159850,2025-07-27,Men Discus Throw (2kg),"YEE, CHUN WAI, ERIC",26,Singapore,45.95,NaN,International - 7de schilderwerken j&M De Coen...,1999,3/7pos (44.98m 45.51m 45.90m 44.32m 45.75m 45....
144118,159851,2025-07-30,Men Discus Throw (2kg),"YEE, CHUN WAI, ERIC",26,Singapore,47.57,NaN,International - Memorial Marcel Soetens + Open...,1999,2/5pos (X 45.03m 47.57m 45.56m 47.52m 44.47m)
144119,159852,2025-07-30,Women High Jump,"SNG, MICHELLE",38,Singapore,1.61,NaN,International - Memorial Marcel Soetens + Open...,1987,1/1pos (1.51 O 1.56 O 1.61 XXO 1.66 X- 1.70 XX)
144120,159853,2025-07-15,Women 200 Meter Dash,"PEREIRA, SHANTI VERONICA",29,Singapore,23.44,0.0,International - Spitzen Leichtathletik Luzern,1996,"Final 1, 8/8pos"


In [708]:
athletes = athletes.drop('wdt_ID', axis=1)

athletes = athletes.rename(columns={'newcolumn': 'DATE', 'newcolumn1': 'EVENT', 'newcolumn2': 'NAME', 'newcolumn3': 'AGE', 'team': 'TEAM', 
                              'result': 'RESULT', 'windms': 'WIND', 'competition': 'COMPETITION', 'yeardob': 'DOB'})

In [709]:
# Remove special characters
    
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()



In [710]:
# DATE column to contain timezone - tz aware mode

athletes['DATE'] = pd.to_datetime(athletes['DATE'], format='mixed', dayfirst=False, utc=True)



In [711]:
# Choose relevant date range for results

athletes['DATE']=athletes['DATE'].dt.tz_localize(None)  # switch off timezone for compatibility with np.datetime64


start = datetime.datetime(2010, 1, 1)
#start = datetime.datetime(2023, 12, 1)


end = datetime.datetime(2022, 12, 31)

start_date = np.datetime64(start)
end_date = np.datetime64(end)


mask = (athletes['DATE'] >= start_date) & (athletes['DATE'] <= end_date)
athletes_selected = athletes.loc[mask]



In [712]:
# Populate year column

athletes_selected['YEAR'] = athletes_selected['DATE'].dt.year


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/1285880273.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['YEAR'] = athletes_selected['DATE'].dt.year


In [713]:
athletes_selected

,DATE,EVENT,NAME,AGE,TEAM,RESULT,WIND,COMPETITION,DOB,infoifany,YEAR
0,2010-12-18,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.59,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010
1,2010-11-21,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.67,-0.1,International - Asian Games,1987,Open Heats 3 Guangdong Olympic Stadium,2010
2,2010-12-18,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.69,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010
3,2010-11-21,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.69,1.2,International - Asian Games,1987,Open Heats 1 Guangdong Olympic Stadium,2010
4,2010-11-22,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.73,0.7,International - Asian Games,1987,Open Semis 3 Guangdong Olympic Stadium,2010
...,...,...,...,...,...,...,...,...,...,...,...
114549,2022-11-20,Women Half Marathon,Rachel See,40,Individual,1:22:07,-,Local - Great Eastern Womens Run,1982,1pos,2022
114591,2022-12-17,Men Marathon,LIM KIEN MAU,42,Individual,2:36:01,-,International - Abu Dhabi Marathon,1981,-,2022
121764,2022-08-27,Women Half Marathon,SHARON TAN FANG YU,33,Individual,1:24:46,-,International - Melbourne Sandy Point Half Mar...,1990,-,2022
130922,2022-10-09,Men Marathon,DEVATHAS SATIANATHAN,34,Individual,2:49:34,-,International - Chicago Marathon,1988,-,2022


In [714]:
mask = athletes_selected['EVENT'].str.contains(r'Men', na=True)
athletes_selected.loc[mask, 'GENDER'] = 'Male'

mask = athletes_selected['EVENT'].str.contains(r'Women', na=True)
athletes_selected.loc[mask, 'GENDER'] = 'Female'

mask = athletes_selected['EVENT'].str.contains(r'Mixed', na=True)
athletes_selected.loc[mask, 'GENDER'] = 'Mixed'


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/277601313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected.loc[mask, 'GENDER'] = 'Male'


In [715]:
athletes_selected['GENDER']

0           Male
1           Male
2           Male
3           Male
4           Male
           ...  
114549    Female
114591      Male
121764    Female
130922      Male
137224      Male
Name: GENDER, Length: 108246, dtype: object

In [716]:
athletes_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108246 entries, 0 to 137224
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE         108246 non-null  datetime64[ns]
 1   EVENT        108246 non-null  object        
 2   NAME         108246 non-null  object        
 3   AGE          108246 non-null  object        
 4   TEAM         108246 non-null  object        
 5   RESULT       108246 non-null  object        
 6   WIND         108246 non-null  object        
 7   COMPETITION  108246 non-null  object        
 8   DOB          108246 non-null  object        
 9   infoifany    108246 non-null  object        
 10  YEAR         108246 non-null  int32         
 11  GENDER       108246 non-null  object        
dtypes: datetime64[ns](1), int32(1), object(10)
memory usage: 10.3+ MB


In [717]:
mask = athletes_selected['COMPETITION'].str.contains(r'International', na=True)
athletes_selected.loc[mask, 'REGION'] = 'International'

mask = athletes_selected['COMPETITION'].str.contains(r'Local', na=True)
athletes_selected.loc[mask, 'REGION'] = 'Local'


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/3756449314.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected.loc[mask, 'REGION'] = 'International'


In [718]:
athletes_selected

,DATE,EVENT,NAME,AGE,TEAM,RESULT,WIND,COMPETITION,DOB,infoifany,YEAR,GENDER,REGION
0,2010-12-18,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.59,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International
1,2010-11-21,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.67,-0.1,International - Asian Games,1987,Open Heats 3 Guangdong Olympic Stadium,2010,Male,International
2,2010-12-18,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.69,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International
3,2010-11-21,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.69,1.2,International - Asian Games,1987,Open Heats 1 Guangdong Olympic Stadium,2010,Male,International
4,2010-11-22,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.73,0.7,International - Asian Games,1987,Open Semis 3 Guangdong Olympic Stadium,2010,Male,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114549,2022-11-20,Women Half Marathon,Rachel See,40,Individual,1:22:07,-,Local - Great Eastern Womens Run,1982,1pos,2022,Female,Local
114591,2022-12-17,Men Marathon,LIM KIEN MAU,42,Individual,2:36:01,-,International - Abu Dhabi Marathon,1981,-,2022,Male,International
121764,2022-08-27,Women Half Marathon,SHARON TAN FANG YU,33,Individual,1:24:46,-,International - Melbourne Sandy Point Half Mar...,1990,-,2022,Female,International
130922,2022-10-09,Men Marathon,DEVATHAS SATIANATHAN,34,Individual,2:49:34,-,International - Chicago Marathon,1988,-,2022,Male,International


In [719]:
mask = athletes_selected['infoifany'].str.contains(r'Final|final|Finals', na=True)
athletes_selected.loc[mask, 'STAGE'] = 'Finals'

mask = athletes_selected['infoifany'].str.contains(r'Prelim', na=True)
athletes_selected.loc[mask, 'STAGE'] = 'Prelim'

mask = athletes_selected['infoifany'].str.contains(r'Semifinal|Semi-final|Semi-Final', na=True)
athletes_selected.loc[mask, 'STAGE'] = 'SF'



mask = athletes_selected['infoifany'].str.contains(r'Semi', na=True)
athletes_selected.loc[mask, 'STAGE'] = 'SF'

mask = athletes_selected['infoifany'].str.contains(r'Heat|Heats|h1|h2|h3|h4|h5|h6|h7|h8|h9', na=True)
athletes_selected.loc[mask, 'STAGE'] = 'Heats'
'''

# Normalize the source text (optional but helps with odd unicode spaces)
info = athletes_selected['infoifany'].astype(str).str.replace(r'[\u00A0\u2000-\u200B]', ' ', regex=True)

# Capture the number that follows "Heat" allowing any non-word separators (space, hyphen, comma, etc.)
num = info.str.extract(r'(?i)\bheat\W*(\d+)')[0]   # case-insensitive, group 1 is the digits
num1 = info.str.extract(r'(?i)\bPrelim\W*(\d+)')[0]   # case-insensitive, group 1 is the digits
num2 = info.str.extract(r'(?i)\bSemis\W*(\d+)')[0]   # case-insensitive, group 1 is the digits
num3 = info.str.extract(r'(?i)\bSemi\W*(\d+)')[0]   # case-insensitive, group 1 is the digits



# Write back "Prelim/Heat <n>" only where we found a number
athletes_selected.loc[num.notna(), 'STAGE'] = 'Heat ' + num
athletes_selected.loc[num1.notna(), 'STAGE'] = 'Prelim ' + num1
athletes_selected.loc[num2.notna(), 'STAGE'] = 'SF ' + num2
athletes_selected.loc[num3.notna(), 'STAGE'] = 'SF ' + num3

'''


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/970888167.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected.loc[mask, 'STAGE'] = 'Finals'


'\n\n# Normalize the source text (optional but helps with odd unicode spaces)\ninfo = athletes_selected[\'infoifany\'].astype(str).str.replace(r\'[\xa0\u2000-\u200b]\', \' \', regex=True)\n\n# Capture the number that follows "Heat" allowing any non-word separators (space, hyphen, comma, etc.)\nnum = info.str.extract(r\'(?i)\x08heat\\W*(\\d+)\')[0]   # case-insensitive, group 1 is the digits\nnum1 = info.str.extract(r\'(?i)\x08Prelim\\W*(\\d+)\')[0]   # case-insensitive, group 1 is the digits\nnum2 = info.str.extract(r\'(?i)\x08Semis\\W*(\\d+)\')[0]   # case-insensitive, group 1 is the digits\nnum3 = info.str.extract(r\'(?i)\x08Semi\\W*(\\d+)\')[0]   # case-insensitive, group 1 is the digits\n\n\n\n# Write back "Prelim/Heat <n>" only where we found a number\nathletes_selected.loc[num.notna(), \'STAGE\'] = \'Heat \' + num\nathletes_selected.loc[num1.notna(), \'STAGE\'] = \'Prelim \' + num1\nathletes_selected.loc[num2.notna(), \'STAGE\'] = \'SF \' + num2\nathletes_selected.loc[num3.notna(

In [720]:
athletes_selected

,DATE,EVENT,NAME,AGE,TEAM,RESULT,WIND,COMPETITION,DOB,infoifany,YEAR,GENDER,REGION,STAGE
0,2010-12-18,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.59,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International,Finals
1,2010-11-21,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.67,-0.1,International - Asian Games,1987,Open Heats 3 Guangdong Olympic Stadium,2010,Male,International,Heats
2,2010-12-18,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.69,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International,Finals
3,2010-11-21,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.69,1.2,International - Asian Games,1987,Open Heats 1 Guangdong Olympic Stadium,2010,Male,International,Heats
4,2010-11-22,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.73,0.7,International - Asian Games,1987,Open Semis 3 Guangdong Olympic Stadium,2010,Male,International,SF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114549,2022-11-20,Women Half Marathon,Rachel See,40,Individual,1:22:07,-,Local - Great Eastern Womens Run,1982,1pos,2022,Female,Local,NaN
114591,2022-12-17,Men Marathon,LIM KIEN MAU,42,Individual,2:36:01,-,International - Abu Dhabi Marathon,1981,-,2022,Male,International,NaN
121764,2022-08-27,Women Half Marathon,SHARON TAN FANG YU,33,Individual,1:24:46,-,International - Melbourne Sandy Point Half Mar...,1990,-,2022,Female,International,NaN
130922,2022-10-09,Men Marathon,DEVATHAS SATIANATHAN,34,Individual,2:49:34,-,International - Chicago Marathon,1988,-,2022,Male,International,NaN


In [721]:
mask = athletes_selected['infoifany'].str.contains(r'U23', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U23'


mask = athletes_selected['infoifany'].str.contains(r'U21', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U21'

mask = athletes_selected['infoifany'].str.contains(r'U20', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U20'

mask = athletes_selected['infoifany'].str.contains(r'U18', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U18'

mask = athletes_selected['infoifany'].str.contains(r'OPEN|Open', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'OPEN'

mask = athletes_selected['infoifany'].str.contains(r'U16', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U16'


mask = athletes_selected['infoifany'].str.contains(r'U15', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U15'

mask = athletes_selected['infoifany'].str.contains(r'U14', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U14'

mask = athletes_selected['infoifany'].str.contains(r'U13', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U13'

mask = athletes_selected['infoifany'].str.contains(r'U23', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U23'

mask = athletes_selected['infoifany'].str.contains(r'U16', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'U16'

mask = athletes_selected['infoifany'].str.contains(r'Masters|masters|MASTERS', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'Masters'

mask = athletes_selected['infoifany'].str.contains(r'A DIVISION|A Division', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'A'

mask = athletes_selected['infoifany'].str.contains(r'B DIVISION|B Division', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'B'

mask = athletes_selected['infoifany'].str.contains(r'C DIVISION|C Division', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'C'

mask = athletes_selected['infoifany'].str.contains(r'OPENS DIVISION|Open Division', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'OPEN'

mask = athletes_selected['infoifany'].str.contains(r'A Boys|A-Boys|A Girls|A-Girls|A-boys|A-girls|A Div', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'A'

mask = athletes_selected['infoifany'].str.contains(r'B Boys|B-Boys|B Girls|B-Girls|B-boys|B-girls|B Div', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'B'

mask = athletes_selected['infoifany'].str.contains(r'C Boys|C-Boys|C Girls|C-Girls|C-boys|C-girls|C Div', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'C'

mask = athletes_selected['infoifany'].str.contains(r'Junior', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'Junior'

mask = athletes_selected['infoifany'].str.contains(r'Youth', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'Youth'

mask = athletes_selected['infoifany'].str.contains(r'Intermediate', na=True)
athletes_selected.loc[mask, 'DIVISION'] = 'Intermediate'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/1567141218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected.loc[mask, 'DIVISION'] = 'U23'


In [722]:
# Extract session details
#athletes_selected['SESSION_CODE'] = (
#    athletes_selected['infoifany']
##    .str.extract(r'Session\s+(\d{4})', flags=re.IGNORECASE)[0]  # Extract '1403'
#    .apply(lambda x: f"S{x[:2]}-{x[2:]}" if pd.notna(x) else None)  # Format S14-03
#)

In [723]:
def extract_session_code(text):
    if pd.isna(text):
        return None
    
    # Case 1: Already in Sxx-xx format → return as is
    match_existing = re.search(r'\bS\d{2}-\d{2}\b', text, flags=re.IGNORECASE)
    if match_existing:
        return match_existing.group(0).upper()
    
    # Case 2: Session #### → format into Sxx-xx
    match_session = re.search(r'Session\s+(\d{4})', text, flags=re.IGNORECASE)
    if match_session:
        digits = match_session.group(1)
        return f"S{digits[:2]}-{digits[2:]}"
    
    return None

In [724]:
athletes_selected['SESSION'] = athletes_selected['infoifany'].apply(extract_session_code)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/4136107182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['SESSION'] = athletes_selected['infoifany'].apply(extract_session_code)


In [725]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')


athletes_selected.to_csv('check.csv', encoding='utf-8')

In [726]:
'''

mask = athletes['Info, if any'].str.contains(r'PB', na=True)
athletes.loc[mask, 'REMARKS'] = 'PB'

mask = athletes['Info, if any'].str.contains(r'SB', na=True)
athletes.loc[mask, 'REMARKS'] = 'SB'

mask = athletes['Info, if any'].str.contains(r'CR', na=True)
athletes.loc[mask, 'REMARKS'] = 'CR'

mask = athletes['Info, if any'].str.contains(r'NR', na=True)
athletes.loc[mask, 'REMARKS'] = 'NR'

mask = athletes['Info, if any'].str.contains(r'R1', na=True)
athletes.loc[mask, 'REMARKS'] = 'R1'

'''


"\n\nmask = athletes['Info, if any'].str.contains(r'PB', na=True)\nathletes.loc[mask, 'REMARKS'] = 'PB'\n\nmask = athletes['Info, if any'].str.contains(r'SB', na=True)\nathletes.loc[mask, 'REMARKS'] = 'SB'\n\nmask = athletes['Info, if any'].str.contains(r'CR', na=True)\nathletes.loc[mask, 'REMARKS'] = 'CR'\n\nmask = athletes['Info, if any'].str.contains(r'NR', na=True)\nathletes.loc[mask, 'REMARKS'] = 'NR'\n\nmask = athletes['Info, if any'].str.contains(r'R1', na=True)\nathletes.loc[mask, 'REMARKS'] = 'R1'\n\n"

In [727]:
# Running

mask = athletes_selected['EVENT'].str.contains(r'1600 Yard Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1600 Yrds'


mask = athletes_selected['EVENT'].str.contains(r'40 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '40m'

mask = athletes_selected['EVENT'].str.contains(r'50 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '50m'
mask = athletes_selected['EVENT'].str.contains(r'60 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '60m'
mask = athletes_selected['EVENT'].str.contains(r'80 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = athletes_selected['EVENT'].str.contains(r'100 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes_selected['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes_selected['EVENT'].str.contains(r'120 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '120m'


mask = athletes_selected['EVENT'].str.contains(r'^100m$', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes_selected['EVENT'].str.contains(r'200 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes_selected['EVENT'].str.contains(r'^200m$', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes_selected['EVENT'].str.contains(r'200\sMeter', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes_selected['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '300m'

mask = athletes_selected['EVENT'].str.contains(r'300 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '300m'

mask = athletes_selected['EVENT'].str.contains(r'400 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes_selected['EVENT'].str.contains(r'^400m$', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes_selected['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes_selected['EVENT'].str.contains(r'500 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '500m'

mask = athletes_selected['EVENT'].str.contains(r'600 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = athletes_selected['EVENT'].str.contains(r'800 Meter Dash', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes_selected['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes_selected['EVENT'].str.contains(r'^800m$', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes_selected['EVENT'].str.contains(r'1000 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = athletes_selected['EVENT'].str.contains(r'1500 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes_selected['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes_selected['EVENT'].str.contains(r'2400 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '3000m'

mask = athletes_selected['EVENT'].str.contains(r'3000 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes_selected['EVENT'].str.contains(r'3000m', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes_selected['EVENT'].str.contains(r'5000 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes_selected['EVENT'].str.contains(r'5 Kilometer Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes_selected['EVENT'].str.contains(r'8 Kilometer Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '8000m'

mask = athletes_selected['EVENT'].str.contains(r'5000m', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes_selected['EVENT'].str.contains(r'10000 Meter Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes_selected['EVENT'].str.contains(r'^10000m$', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes_selected['EVENT'].str.contains(r'10 Kilometer Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '10,000m'

mask = athletes_selected['EVENT'].str.contains(r'1 Mile Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1 mile'
mask = athletes_selected['EVENT'].str.contains(r'Mile Run', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1 mile'


mask = athletes_selected['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '10,000m'



# Hurdles


mask = athletes_selected['EVENT'].str.contains(r'60 Meter Hurdles', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '60m Hurdles'

mask = athletes_selected['EVENT'].str.contains(r'80\sMeter\sHurdles', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'

mask = athletes_selected['EVENT'].str.contains(r'100 Meter Hurdles', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


mask = athletes_selected['EVENT'].str.contains(r'110 Meter Hurdles', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

mask = athletes_selected['EVENT'].str.contains(r'200 Meter Hurdles', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '200m Hurdles'

mask = athletes_selected['EVENT'].str.contains(r'300 Meter Hurdles', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '300m Hurdles'

mask = athletes_selected['EVENT'].str.contains(r'400 Meter Hurdles', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


# Throws

mask = athletes_selected['EVENT'].str.contains(r'Javelin\sThrow', na=True, regex=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = athletes_selected['EVENT'].str.contains(r'Shot\sPut', na=True, regex=True) # there are some additional characters after Put
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = athletes_selected['EVENT'].str.contains(r'Hammer\sThrow', na=True)  # there are some characters after Throw
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'


mask = athletes_selected['EVENT'].str.contains(r'Discus\sThrow', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes_selected['EVENT'].str.contains(r'High Jump', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes_selected['EVENT'].str.contains(r'Long\sJump', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes_selected['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes_selected['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes_selected['EVENT'].str.contains(r'Triple Jump', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes_selected['EVENT'].str.contains(r'Pole Vault', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes_selected['EVENT'].str.contains(r'High jump', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes_selected['EVENT'].str.contains(r'Long jump', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes_selected['EVENT'].str.contains(r'Triple jump', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes_selected['EVENT'].str.contains(r'Pole\svault', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

mask = athletes_selected['EVENT'].str.contains(r'2000m S/C', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes_selected['EVENT'].str.contains(r'2000m Steeplechase', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes_selected['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes_selected['EVENT'].str.contains(r'3000m S/C', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = athletes_selected['EVENT'].str.contains(r'3000 Meter Steeplechase', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'

# Marathon

mask = athletes_selected['EVENT'].str.contains(r'Marathon', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes_selected['EVENT'].str.contains(r'Half\sMarathon', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'

# Walk

mask = athletes_selected['EVENT'].str.contains(r'800 Meter Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '800m Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'1 Kilometer Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1km Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'3 Kilometer Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '3km Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'5 Kilometer Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '5km Racewalk'

mask = athletes_selected['EVENT'].str.contains(r'1500m Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1500m Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1500m Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'20000 Meter Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '2000m Racewalk'


mask = athletes_selected['EVENT'].str.contains(r'3000m Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '3000m Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '3000m Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'5000m Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '10,000m Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'10 Kilometer Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '10km Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'20 Kilometer Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '20km Racewalk'
mask = athletes_selected['EVENT'].str.contains(r'50 Kilometer Race Walk', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '50km Racewalk'


# Relay


mask = athletes_selected['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '4 x 80m Relay'
mask = athletes_selected['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '4 x 100m Relay'
mask = athletes_selected['EVENT'].str.contains(r'4x400 Meter Relay', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '4 x 400m Relay'
mask = athletes_selected['EVENT'].str.contains(r'8x80 Meter Relay', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '4 x 80m Relay'
mask = athletes_selected['EVENT'].str.contains(r'4x800 Meter Relay', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '4 x 800m Relay'
mask = athletes_selected['EVENT'].str.contains(r'2x800 Meter Relay (2x2)', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '2 x 800m Relay(2x2)'

mask = athletes_selected['EVENT'].str.contains(r'2x800 Meter Relay', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '2 x 800m Relay(2x2)'


mask = athletes_selected['EVENT'].str.contains(r'Sprint Medley Relay', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Sprint Medley Relay'
mask = athletes_selected['EVENT'].str.contains(r'Distance Medley Relay', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Distance Medley Relay'
mask = athletes_selected['EVENT'].str.contains(r'2x800 Meter Relay (2x2)', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '2 x 800m Meter Relay(2x2)'

mask = athletes_selected['EVENT'].str.contains(r'4x100m', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Shuttle Hurdles Relay(4x100m)'
mask = athletes_selected['EVENT'].str.contains(r'4x110m', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Shuttle Hurdles Relay(4x110m)'
mask = athletes_selected['EVENT'].str.contains(r'4x80m', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Shuttle Hurdles Relay(4x80m)'


# Decathlon/Heptathlon

mask = athletes_selected['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes_selected['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes_selected['EVENT'].str.contains(r'Heptathlon', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes_selected['EVENT'].str.contains(r'Decathlon', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes_selected['EVENT'].str.contains(r'Octathlon', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Octathlon'
mask = athletes_selected['EVENT'].str.contains(r'Pentathlon', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Pentathlon'



# Cross Country

mask = athletes_selected['EVENT'].str.contains(r'10 Kilometer Run (Cross Country)', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = '10km Cross Country'

# Road

mask = athletes_selected['EVENT'].str.contains(r'Mile Road', na=True)
athletes_selected.loc[mask, 'MAPPED_EVENT'] = 'Mile Road'




/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/1078943859.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected.loc[mask, 'MAPPED_EVENT'] = '1600 Yrds'
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/1078943859.py:237: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = athletes_selected['EVENT'].str.contains(r'2x800 Meter Relay (2x2)', na=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/1078943859.py:248: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = athletes_selected['EVENT'].str.contains(r

In [728]:
athletes_selected

,DATE,EVENT,NAME,AGE,TEAM,RESULT,WIND,COMPETITION,DOB,infoifany,YEAR,GENDER,REGION,STAGE,DIVISION,SESSION,MAPPED_EVENT
0,2010-12-18,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.59,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International,Finals,OPEN,None,100m
1,2010-11-21,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.67,-0.1,International - Asian Games,1987,Open Heats 3 Guangdong Olympic Stadium,2010,Male,International,Heats,OPEN,None,100m
2,2010-12-18,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.69,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International,Finals,OPEN,None,100m
3,2010-11-21,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.69,1.2,International - Asian Games,1987,Open Heats 1 Guangdong Olympic Stadium,2010,Male,International,Heats,OPEN,None,100m
4,2010-11-22,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.73,0.7,International - Asian Games,1987,Open Semis 3 Guangdong Olympic Stadium,2010,Male,International,SF,OPEN,None,100m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114549,2022-11-20,Women Half Marathon,Rachel See,40,Individual,1:22:07,-,Local - Great Eastern Womens Run,1982,1pos,2022,Female,Local,NaN,NaN,None,Half Marathon
114591,2022-12-17,Men Marathon,LIM KIEN MAU,42,Individual,2:36:01,-,International - Abu Dhabi Marathon,1981,-,2022,Male,International,NaN,NaN,None,Marathon
121764,2022-08-27,Women Half Marathon,SHARON TAN FANG YU,33,Individual,1:24:46,-,International - Melbourne Sandy Point Half Mar...,1990,-,2022,Female,International,NaN,NaN,None,Half Marathon
130922,2022-10-09,Men Marathon,DEVATHAS SATIANATHAN,34,Individual,2:49:34,-,International - Chicago Marathon,1988,-,2022,Male,International,NaN,NaN,None,Marathon


In [729]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')


athletes_selected.to_csv('check.csv', encoding='utf-8')

In [730]:
# Extract ranking

athletes_selected['RANK'] = athletes_selected['infoifany'].str.extract(r'(\d+)\s*(?=pos)', flags=re.IGNORECASE)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/1269971243.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['RANK'] = athletes_selected['infoifany'].str.extract(r'(\d+)\s*(?=pos)', flags=re.IGNORECASE)


In [731]:
'''
1. (\d+)
\d → means “digit” (0–9)

+ → means “one or more of the preceding thing”

Together \d+ means one or more digits

The parentheses (...) → create a capturing group, which is the part you want to return in .str.extract()

Example: In "10pos", this captures "10"

In "1 pos", this captures "1"

2. \s*
\s → means “whitespace” (spaces, tabs, newlines)

* → means “zero or more of the preceding thing”

Together \s* means allow for any number of spaces between the digits and "pos"

Example: "1pos" → zero spaces, "1 pos" → one space, both match

3. (?=pos)
This is a positive lookahead

It checks that immediately after whatever we just matched, the letters "pos" appear

Crucially, it does not consume "pos" — meaning "pos" is not part of the match, it just has to be there

Example:

"10pos" → matches 10 because "pos" follows it

"10 position" → matches 10 because "pos" starts "position"

"10abc" → does not match because "pos" is not there

4. flags=re.IGNORECASE
Makes the whole match case-insensitive, so it will match:

"pos", "Pos", "POS", "pOs", etc.

Putting it together:
The regex says:

"Find one or more digits, optionally followed by spaces, only if the next letters are "pos" (in any capitalization). Capture just the digits."

Example walkthrough with "3/8pos, final"

The regex engine scans left to right

Finds 3 → matches (\d+) → capture group = "3"

Looks ahead: next characters are /8pos → does match "pos" eventually, because the lookahead is positioned right after the digits and the / stops this match — that’s why it only gets the "3", not "3/8".

If you wanted "3/8" instead, you’d need a different pattern like:

regex
Copy
Edit
([\d/]+)(?=pos)
This says "digits or /, one or more, before pos".
'''


'\n1. (\\d+)\n\\d → means “digit” (0–9)\n\n+ → means “one or more of the preceding thing”\n\nTogether \\d+ means one or more digits\n\nThe parentheses (...) → create a capturing group, which is the part you want to return in .str.extract()\n\nExample: In "10pos", this captures "10"\n\nIn "1 pos", this captures "1"\n\n2. \\s*\n\\s → means “whitespace” (spaces, tabs, newlines)\n\n* → means “zero or more of the preceding thing”\n\nTogether \\s* means allow for any number of spaces between the digits and "pos"\n\nExample: "1pos" → zero spaces, "1 pos" → one space, both match\n\n3. (?=pos)\nThis is a positive lookahead\n\nIt checks that immediately after whatever we just matched, the letters "pos" appear\n\nCrucially, it does not consume "pos" — meaning "pos" is not part of the match, it just has to be there\n\nExample:\n\n"10pos" → matches 10 because "pos" follows it\n\n"10 position" → matches 10 because "pos" starts "position"\n\n"10abc" → does not match because "pos" is not there\n\n4. f

In [732]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')


athletes_selected.to_csv('check.csv', encoding='utf-8')

In [733]:
'''
# Extract date

def date(string):
    
    
    try:
        
        slash=re.search(r'2025', string)
        
        end = slash.start()
    
        date = string[:end]
            
    except:
        
        date = ''
    
    return date
    

#athletes['DATE'] = athletes['Date'].apply(date)
'''

"\n# Extract date\n\ndef date(string):\n    \n    \n    try:\n        \n        slash=re.search(r'2025', string)\n        \n        end = slash.start()\n    \n        date = string[:end]\n            \n    except:\n        \n        date = ''\n    \n    return date\n    \n\n#athletes['DATE'] = athletes['Date'].apply(date)\n"

In [734]:
athletes_selected

,DATE,EVENT,NAME,AGE,TEAM,RESULT,WIND,COMPETITION,DOB,infoifany,YEAR,GENDER,REGION,STAGE,DIVISION,SESSION,MAPPED_EVENT,RANK
0,2010-12-18,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.59,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International,Finals,OPEN,None,100m,NaN
1,2010-11-21,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.67,-0.1,International - Asian Games,1987,Open Heats 3 Guangdong Olympic Stadium,2010,Male,International,Heats,OPEN,None,100m,NaN
2,2010-12-18,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.69,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International,Finals,OPEN,None,100m,NaN
3,2010-11-21,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.69,1.2,International - Asian Games,1987,Open Heats 1 Guangdong Olympic Stadium,2010,Male,International,Heats,OPEN,None,100m,NaN
4,2010-11-22,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.73,0.7,International - Asian Games,1987,Open Semis 3 Guangdong Olympic Stadium,2010,Male,International,SF,OPEN,None,100m,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114549,2022-11-20,Women Half Marathon,Rachel See,40,Individual,1:22:07,-,Local - Great Eastern Womens Run,1982,1pos,2022,Female,Local,NaN,NaN,None,Half Marathon,1
114591,2022-12-17,Men Marathon,LIM KIEN MAU,42,Individual,2:36:01,-,International - Abu Dhabi Marathon,1981,-,2022,Male,International,NaN,NaN,None,Marathon,NaN
121764,2022-08-27,Women Half Marathon,SHARON TAN FANG YU,33,Individual,1:24:46,-,International - Melbourne Sandy Point Half Mar...,1990,-,2022,Female,International,NaN,NaN,None,Half Marathon,NaN
130922,2022-10-09,Men Marathon,DEVATHAS SATIANATHAN,34,Individual,2:49:34,-,International - Chicago Marathon,1988,-,2022,Male,International,NaN,NaN,None,Marathon,NaN


In [735]:
# Extract heat

def heat(string):
    
    
    try:
        
        heat=re.findall(r'h\d|H\d|h\d\d|H\d\d', string)
        
        heat1=heat[0]
                
            
    except:
        
        heat1 = ''
    
    return heat1
    

athletes_selected['HEAT'] = athletes_selected['infoifany'].apply(heat)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/3075324093.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['HEAT'] = athletes_selected['infoifany'].apply(heat)


In [736]:
# Check output

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')

athletes_selected.to_csv("test.csv", index=False, encoding="utf-8")

In [737]:
athletes_selected

,DATE,EVENT,NAME,AGE,TEAM,RESULT,WIND,COMPETITION,DOB,infoifany,YEAR,GENDER,REGION,STAGE,DIVISION,SESSION,MAPPED_EVENT,RANK,HEAT
0,2010-12-18,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.59,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International,Finals,OPEN,None,100m,NaN,
1,2010-11-21,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.67,-0.1,International - Asian Games,1987,Open Heats 3 Guangdong Olympic Stadium,2010,Male,International,Heats,OPEN,None,100m,NaN,
2,2010-12-18,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.69,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",2010,Male,International,Finals,OPEN,None,100m,NaN,
3,2010-11-21,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.69,1.2,International - Asian Games,1987,Open Heats 1 Guangdong Olympic Stadium,2010,Male,International,Heats,OPEN,None,100m,NaN,
4,2010-11-22,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.73,0.7,International - Asian Games,1987,Open Semis 3 Guangdong Olympic Stadium,2010,Male,International,SF,OPEN,None,100m,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114549,2022-11-20,Women Half Marathon,Rachel See,40,Individual,1:22:07,-,Local - Great Eastern Womens Run,1982,1pos,2022,Female,Local,NaN,NaN,None,Half Marathon,1,
114591,2022-12-17,Men Marathon,LIM KIEN MAU,42,Individual,2:36:01,-,International - Abu Dhabi Marathon,1981,-,2022,Male,International,NaN,NaN,None,Marathon,NaN,
121764,2022-08-27,Women Half Marathon,SHARON TAN FANG YU,33,Individual,1:24:46,-,International - Melbourne Sandy Point Half Mar...,1990,-,2022,Female,International,NaN,NaN,None,Half Marathon,NaN,
130922,2022-10-09,Men Marathon,DEVATHAS SATIANATHAN,34,Individual,2:49:34,-,International - Chicago Marathon,1988,-,2022,Male,International,NaN,NaN,None,Marathon,NaN,


In [738]:
# Extract qualitication

def qual(string):
    
    if string == 'Q':

        qual1 = 'Q'
        
    elif string == 'q':

        qual1 = 'q'
        
    else:
    
        try:
    
            qual=re.findall(r'\sq|\sQ|\sq\s|\sQ\s', string)
            qual1=qual[0]        
                    
                
        except:
                
            qual1 = ''
        
    return qual1
    

athletes_selected['QUALIFICATION'] = athletes_selected['infoifany'].apply(qual)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/3195856429.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['QUALIFICATION'] = athletes_selected['infoifany'].apply(qual)


In [739]:
# Check output

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')

athletes_selected.to_csv("test.csv", index=False, encoding="utf-8")

In [740]:
# Extract event class

def event_class(string):
    
    
    try:
        
        left=re.search(r'\(', string)
        right=re.search(r'\)', string)
        
        
        type = string[left.start()+1:right.start()]
        
                 
    except:
        
        type = ''
    
    return type
    

athletes_selected['EVENT_CLASS'] = athletes_selected['EVENT'].apply(event_class)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/2786521242.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['EVENT_CLASS'] = athletes_selected['EVENT'].apply(event_class)


In [741]:
# Check output

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')

athletes_selected.to_csv("test.csv", index=False, encoding="utf-8")

In [742]:
athletes_selected

,DATE,EVENT,NAME,AGE,TEAM,RESULT,WIND,COMPETITION,DOB,infoifany,...,GENDER,REGION,STAGE,DIVISION,SESSION,MAPPED_EVENT,RANK,HEAT,QUALIFICATION,EVENT_CLASS
0,2010-12-18,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.59,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",...,Male,International,Finals,OPEN,None,100m,NaN,,,
1,2010-11-21,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.67,-0.1,International - Asian Games,1987,Open Heats 3 Guangdong Olympic Stadium,...,Male,International,Heats,OPEN,None,100m,NaN,,,
2,2010-12-18,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.69,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",...,Male,International,Finals,OPEN,None,100m,NaN,,,
3,2010-11-21,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.69,1.2,International - Asian Games,1987,Open Heats 1 Guangdong Olympic Stadium,...,Male,International,Heats,OPEN,None,100m,NaN,,,
4,2010-11-22,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.73,0.7,International - Asian Games,1987,Open Semis 3 Guangdong Olympic Stadium,...,Male,International,SF,OPEN,None,100m,NaN,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114549,2022-11-20,Women Half Marathon,Rachel See,40,Individual,1:22:07,-,Local - Great Eastern Womens Run,1982,1pos,...,Female,Local,NaN,NaN,None,Half Marathon,1,,,
114591,2022-12-17,Men Marathon,LIM KIEN MAU,42,Individual,2:36:01,-,International - Abu Dhabi Marathon,1981,-,...,Male,International,NaN,NaN,None,Marathon,NaN,,,
121764,2022-08-27,Women Half Marathon,SHARON TAN FANG YU,33,Individual,1:24:46,-,International - Melbourne Sandy Point Half Mar...,1990,-,...,Female,International,NaN,NaN,None,Half Marathon,NaN,,,
130922,2022-10-09,Men Marathon,DEVATHAS SATIANATHAN,34,Individual,2:49:34,-,International - Chicago Marathon,1988,-,...,Male,International,NaN,NaN,None,Marathon,NaN,,,


In [743]:
athletes_selected['COMPETITION_CLEAN'] = (
    athletes_selected['COMPETITION']
    .str.replace(r'^(International|Local)\s*-\s*', '', regex=True, flags=re.IGNORECASE)
    .str.strip()
)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/1591122720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['COMPETITION_CLEAN'] = (


In [744]:
# Clean up final text

mask = athletes_selected['COMPETITION_CLEAN'].str.contains(r'National Schools', na=True)
athletes_selected.loc[mask, 'COMPETITION_CLEAN'] = 'National School Games'



In [745]:
# Check output

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')

athletes_selected.to_csv("test.csv", index=False, encoding="utf-8")

In [746]:
#athletes_selected.to_csv("SAA_external_database_2023.csv", encoding="utf-8")

In [747]:
athletes_selected

,DATE,EVENT,NAME,AGE,TEAM,RESULT,WIND,COMPETITION,DOB,infoifany,...,REGION,STAGE,DIVISION,SESSION,MAPPED_EVENT,RANK,HEAT,QUALIFICATION,EVENT_CLASS,COMPETITION_CLEAN
0,2010-12-18,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.59,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",...,International,Finals,OPEN,None,100m,NaN,,,,ASEAN University Games
1,2010-11-21,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.67,-0.1,International - Asian Games,1987,Open Heats 3 Guangdong Olympic Stadium,...,International,Heats,OPEN,None,100m,NaN,,,,Asian Games
2,2010-12-18,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.69,0.4,International - ASEAN University Games,1987,"Open Final Maejo University Main Stadium, Chia...",...,International,Finals,OPEN,None,100m,NaN,,,,ASEAN University Games
3,2010-11-21,Men 100 Meter Dash,Md. Elfi Mustapa,23,Singapore,10.69,1.2,International - Asian Games,1987,Open Heats 1 Guangdong Olympic Stadium,...,International,Heats,OPEN,None,100m,NaN,,,,Asian Games
4,2010-11-22,Men 100 Meter Dash,Md. Amirudin Jamal,23,Singapore,10.73,0.7,International - Asian Games,1987,Open Semis 3 Guangdong Olympic Stadium,...,International,SF,OPEN,None,100m,NaN,,,,Asian Games
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114549,2022-11-20,Women Half Marathon,Rachel See,40,Individual,1:22:07,-,Local - Great Eastern Womens Run,1982,1pos,...,Local,NaN,NaN,None,Half Marathon,1,,,,Great Eastern Womens Run
114591,2022-12-17,Men Marathon,LIM KIEN MAU,42,Individual,2:36:01,-,International - Abu Dhabi Marathon,1981,-,...,International,NaN,NaN,None,Marathon,NaN,,,,Abu Dhabi Marathon
121764,2022-08-27,Women Half Marathon,SHARON TAN FANG YU,33,Individual,1:24:46,-,International - Melbourne Sandy Point Half Mar...,1990,-,...,International,NaN,NaN,None,Half Marathon,NaN,,,,Melbourne Sandy Point Half Marathon
130922,2022-10-09,Men Marathon,DEVATHAS SATIANATHAN,34,Individual,2:49:34,-,International - Chicago Marathon,1988,-,...,International,NaN,NaN,None,Marathon,NaN,,,,Chicago Marathon


In [748]:
#athletes_selected.to_csv("athletes_2023.csv", encoding='utf-8')

In [749]:
# Map general event category 

athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'40m|40 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'60m|60 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'100m|100 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'120m|120 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'400m|400 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'500m|500 Meter Run', na=True), 'CATEGORY_EVENT'] = 'Mid'

athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'600m|600 Meter Run', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'60m|80 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'200m|200 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'50m|50 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'80m|80 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'300m|300 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Mile|mile', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'1600 Yard', na=True), 'CATEGORY_EVENT'] = 'Mid'


athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'5000m|5000 Meter Run', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'2000m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'3000m|3000 Meter Run', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'2400m|2400 Meter Run', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'800m|800 Meter Run', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'1000m|1000 Meter Run', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'1500m|1500 Meter Run', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'10,000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'10000m|10000 Meter Run', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'5km|5 Kilometer Run', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'8km|8 Kilometer Run', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'10km|10 Kilometer Run', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Road', na=True), 'CATEGORY_EVENT'] = 'Road'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'


# Override values

athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'vault', na=True), 'CATEGORY_EVENT'] = 'Jump'

athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'jump', na=True), 'CATEGORY_EVENT'] = 'Jump'

athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'

athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Heptathlon', na=True), 'CATEGORY_EVENT'] = 'Heptathlon'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Octathlon', na=True), 'CATEGORY_EVENT'] = 'Octathlon'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Marathon|marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'Steeplechase', na=True), 'CATEGORY_EVENT'] = 'Steeple'




/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/3943882784.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected.loc[athletes_selected['EVENT'].str.contains(r'40m|40 Meter Dash', na=True), 'CATEGORY_EVENT'] = 'Sprint'


In [750]:
'''
# Extract year

def time(string):
    
    
    try:
        
        slash=re.search(r'AM|PM', string)
        
        end = slash.start()
    
        time = string[:end]
            
    except:
        
        time = ''
    
    return time
    

#athletes['RESULT_CLEANED'] = athletes['RESULT'].apply(time)
'''

"\n# Extract year\n\ndef time(string):\n    \n    \n    try:\n        \n        slash=re.search(r'AM|PM', string)\n        \n        end = slash.start()\n    \n        time = string[:end]\n            \n    except:\n        \n        time = ''\n    \n    return time\n    \n\n#athletes['RESULT_CLEANED'] = athletes['RESULT'].apply(time)\n"

In [751]:
athletes_selected.columns

Index(['DATE', 'EVENT', 'NAME', 'AGE', 'TEAM', 'RESULT', 'WIND', 'COMPETITION',
       'DOB', 'infoifany', 'YEAR', 'GENDER', 'REGION', 'STAGE', 'DIVISION',
       'SESSION', 'MAPPED_EVENT', 'RANK', 'HEAT', 'QUALIFICATION',
       'EVENT_CLASS', 'COMPETITION_CLEAN', 'CATEGORY_EVENT'],
      dtype='object')

In [759]:
# clean up remainders

athletes_selected = athletes_selected.rename(columns={'infoifany': 'REMARKS'})
athletes_selected['COMPETITION']=athletes_selected['COMPETITION_CLEAN']
athletes_selected['EVENT']=athletes_selected['EVENT_MAPPED']


athletes_selected = athletes_selected.drop(['COMPETITION_CLEAN', 'EVENT_MAPPED'], axis=1)


KeyError: 'COMPETITION_CLEAN'

In [753]:
# Set timestamp

import datetime

# Set timestamp

timezone = pytz.timezone('UTC')
dt_ms = datetime.datetime.now().replace(tzinfo=timezone)
athletes_selected.loc[:,'TIMESTAMP'] = dt_ms.replace(second=0, microsecond=0)

# Set date time format

athletes_selected['DATE'] = athletes_selected['DATE'].dt.tz_localize('UTC')


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/737412711.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected.loc[:,'TIMESTAMP'] = dt_ms.replace(second=0, microsecond=0)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/737412711.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['DATE'] = athletes_selected['DATE'].dt.tz_localize('UTC')


In [754]:
# Convert to NEW SCHEMA

athletes_selected['LAST_NAME'] = ''
athletes_selected['FIRST_NAME'] = ''
athletes_selected['OTHER_NAME'] = ''
athletes_selected['SEED'] = ''
athletes_selected['LANE'] = ''
athletes_selected['ATHLETE_ID'] = ''
athletes_selected['TAG_ID'] = ''
athletes_selected['POINTS'] = ''
athletes_selected['GROUP'] = ''
athletes_selected['SESSION']=''
athletes_selected['DISTANCE']=''
athletes_selected['HOST_CITY']=''
athletes_selected['SUB_EVENT']=''
athletes_selected['DICT_RESULTS']=''
athletes_selected['RX_TIME']=''
athletes_selected['VENUE']=''
athletes_selected['UNIQUE_ID']=''
athletes_selected['NATIONALITY']=''
athletes_selected['SOURCE']='SAA Database'

athletes_selected = athletes_selected.reindex(columns= ['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
                                        'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'NATIONALITY',
                                        'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID',
                                        'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE', 'HOST_CITY', 'RX_TIME'])


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/639906320.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['LAST_NAME'] = ''
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/639906320.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_selected['FIRST_NAME'] = ''
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_39195/639906320.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [755]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')


athletes_selected.to_csv("athletes_selected.csv", encoding='utf-8')

In [756]:
len(athletes_selected.columns)

40

In [757]:
athletes_selected

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,,,,Md. Elfi Mustapa,NaN,,Singapore,,10.59,,...,SAA Database,NaN,2025-08-14 22:46:00+00:00,,,,,,,
1,,,,Md. Amirudin Jamal,NaN,,Singapore,,10.67,,...,SAA Database,NaN,2025-08-14 22:46:00+00:00,,,,,,,
2,,,,Md. Amirudin Jamal,NaN,,Singapore,,10.69,,...,SAA Database,NaN,2025-08-14 22:46:00+00:00,,,,,,,
3,,,,Md. Elfi Mustapa,NaN,,Singapore,,10.69,,...,SAA Database,NaN,2025-08-14 22:46:00+00:00,,,,,,,
4,,,,Md. Amirudin Jamal,NaN,,Singapore,,10.73,,...,SAA Database,NaN,2025-08-14 22:46:00+00:00,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114549,,,,Rachel See,1,,Individual,,1:22:07,,...,SAA Database,NaN,2025-08-14 22:46:00+00:00,,,,,,,
114591,,,,LIM KIEN MAU,NaN,,Individual,,2:36:01,,...,SAA Database,NaN,2025-08-14 22:46:00+00:00,,,,,,,
121764,,,,SHARON TAN FANG YU,NaN,,Individual,,1:24:46,,...,SAA Database,NaN,2025-08-14 22:46:00+00:00,,,,,,,
130922,,,,DEVATHAS SATIANATHAN,NaN,,Individual,,2:49:34,,...,SAA Database,NaN,2025-08-14 22:46:00+00:00,,,,,,,


In [758]:
import datetime

gender_pattern = 'Male|Female|Mixed'
year_pattern = '\d\d\d\d'
format_string = "%Y-%m-%d %H:%M:%S%z"


#df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])


df['YEAR'] = df['YEAR'].astype(str)
df['DATE'] = df['DATE'].astype(str)
df['DOB'] = df['DOB'].astype(str)

#df['event_date_dt'] = pd.to_datetime(df['event_date'], format='mixed', dayfirst=False, utc=True)

stage_list = ['Finals', 'Semifinal', 'Heats', 'Qualification', '']

event_list = ['100m', '400m', '800m', '1500m', '1500m Walk', '3000m', '5000m', '10,000m', '80m Hurdles', '110m Hurdles', '100m Hurdles', '200m Hurdles', '400m Hurdles', 
              'High Jump', 'Discus', 'Discus Throw', 'Triple Jump', 'Pole Vault','Long Jump', 'Javelin', 'Javelin Throw', 'Shot Put', 'High Jump', '2000m SC', '3000m Walk', 
              '5000m Walk', 'Hammer Throw', '200m', '3000m SC', '4 x 100m Relay', '4 x 400m Relay'] 
category_list = ['Throw', 'Jump', 'Relay', 'Long', 'Mid', 'Sprint', 'Road', 'Cross Country', 'Steeple', 'Hurdles', 'Walk', 'Marathon', 'Triathlon', 'Decathlon', 'Pentathlon']

for index, row in df.iterrows():
        
    assert re.search(gender_pattern, row['GENDER']) is not None   # check format of gender input
    assert row['NAME'] != None
    #assert len(row['NAME'])>5
    assert row['DATE'] != None
    assert row['EVENT'] != None 
    assert row['COMPETITION'] != None 
    assert (row['REGION'] == 'Local' or row['REGION'] == 'International')
    assert row['DATE'] is not None
    assert row['CATEGORY_EVENT'] in category_list
    assert len(row['DATE'])>3
    assert re.search('\)|\(', row['EVENT']) is None  # make sure there are no brackets in the event column
  #  assert row['NATIONALITY']=='SGP' # For Tilastopaja extractions
    assert re.search(year_pattern, row['YEAR']) is not None   # check format of year
  #  assert row['STAGE'] in stage_list 
  #  assert row['EVENT'] in event_list 
    assert re.search('Para|para', row['EVENT']) is None

    assert datetime.datetime.strptime(row['DATE'], format_string)

    
 #   assert not row['event_date_dt'] is pd.NaT
    
    
    # Check column locations before loading

    assert len(df.columns) == 40
    assert df.columns.get_loc('RESULT') == 8
    assert df.columns.get_loc('AGE') == 17
    assert df.columns.get_loc('WIND') == 12
    assert df.columns.get_loc('COMPETITION')==24
    
   

    # Set timestamp

    timezone = pytz.timezone('UTC')
    dt_ms = datetime.datetime.now().replace(tzinfo=timezone)
    df.loc[:,'TIMESTAMP'] = dt_ms.replace(second=0, microsecond=0)


print('PASSED')

df

NameError: name 'df' is not defined